In [22]:
import pandas as pd

# 1) CSV 읽기
df_17 = pd.read_csv(
    r"C:\Users\jinuk\DAT_ASCII_SPXUSD_M1_2017.csv",
    sep=';',                     # 세미콜론 기준
    header=None,
    names=['datetime','open','high','low','close','volume'],
    dtype={'datetime': str}
)

# 2) '20170102 180000' 형태를 진짜 날짜/시간으로 바꾸기
df_17['datetime'] = pd.to_datetime(df_17['datetime'], format='%Y%m%d %H%M%S')
print(df.head(15))



              datetime     open     high      low    close  volume
0  2016-11-07 00:02:00  2105.50  2105.75  2105.50  2105.75     0.0
1  2016-11-07 00:03:00  2105.75  2106.00  2105.75  2106.00     0.0
2  2016-11-07 00:04:00  2106.25  2106.25  2106.00  2106.00     0.0
3  2016-11-07 00:05:00  2105.75  2106.00  2105.75  2105.75     0.0
4  2016-11-07 00:12:00  2106.00  2106.00  2106.00  2106.00     0.0
5  2016-11-07 00:13:00  2106.25  2106.25  2106.25  2106.25     0.0
6  2016-11-07 00:15:00  2106.50  2106.50  2106.50  2106.50     0.0
7  2016-11-07 00:18:00  2106.75  2106.75  2106.50  2106.50     0.0
8  2016-11-07 00:19:00  2106.25  2106.25  2106.00  2106.00     0.0
9  2016-11-07 00:20:00  2105.75  2105.75  2105.75  2105.75     0.0
10 2016-11-07 00:21:00  2106.00  2106.00  2106.00  2106.00     0.0
11 2016-11-07 00:23:00  2105.75  2106.00  2105.75  2105.75     0.0
12 2016-11-07 00:26:00  2105.50  2105.75  2105.50  2105.75     0.0
13 2016-11-07 00:30:00  2106.00  2106.00  2105.75  2105.75    

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import json, html
from datetime import datetime
import time

url = "https://www.thetrumparchive.com/?resultssortOption=%22Latest%22&startDate=%222017-01-01%22&endDate=%222017-12-31%22"

driver = webdriver.Chrome()
driver.get(url)

# 스크립트 로딩 시간 필요
time.sleep(3)

html_source = driver.page_source
driver.quit()

soup = BeautifulSoup(html_source, "html.parser")
tweets = []

for div in soup.select("div.ttaTweet")[:10]:
    raw = div.get("data-tweet")
    if not raw:
        continue

    data = json.loads(html.unescape(raw))
    dt_utc = datetime.utcfromtimestamp(data["date"] / 1000)

    inner_html = html.unescape(data["text"])
    inner_soup = BeautifulSoup(inner_html, "html.parser")
    text = inner_soup.get_text(" ", strip=True)

    tweets.append({
        "datetime_utc": dt_utc,
        "id": data["id"],
        "text": text,
        "retweets": data["retweets"],
        "favorites": data["favorites"]
    })

for i, t in enumerate(tweets, start=1):
    print(f"[{i}] {t['datetime_utc']} | RT={t['retweets']} ♥={t['favorites']}")
    print(t["text"])
    print("-"*80)


[1] 2017-12-31 04:46:21 | RT=10928 ♥=55336
“Trump Rally: Stocks put 2017 in the record books” https://t.co/0foQGaFjMh
--------------------------------------------------------------------------------
[2] 2017-12-30 22:42:09 | RT=20115 ♥=106921
Jobs are kicking in and companies are coming back to the U.S. Unnecessary regulations and high taxes are being dramatically Cut, and it will only get better. MUCH MORE TO COME!
--------------------------------------------------------------------------------
[3] 2017-12-30 22:36:41 | RT=42893 ♥=179632
I use Social Media not because I like to, but because it is the only way to fight a VERY dishonest and unfair “press,” now often referred to as Fake News Media. Phony and non-existent “sources” are being used more often than ever. Many stories &, reports a pure fiction!
--------------------------------------------------------------------------------
[4] 2017-12-30 21:12:45 | RT=13561 ♥=66759
On Taxes: “This is the biggest corporate rate cut ever, goin

C:\Users\jinuk\AppData\Local\Temp\ipykernel_33216\1840524118.py:27: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt_utc = datetime.utcfromtimestamp(data["date"] / 1000)
C:\Users\jinuk\AppData\Local\Temp\ipykernel_33216\1840524118.py:30: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  inner_soup = BeautifulSoup(inner_html, "html.parser")


In [10]:
import pandas as pd

# 원본 CSV 불러오기 (세미콜론 구분)
df = pd.read_csv(r"C:\Users\jinuk\Downloads\DAT_ASCII_SPXUSD_M1_2016\DAT_ASCII_SPXUSD_M1_2016.csv", sep=";", header=None,
                 names=["raw_datetime", "open", "high", "low", "close", "volume"])

# datetime 분리 및 변환
# "20160103 180100" → "2016-01-03 18:01:00"
df["datetime"] = pd.to_datetime(df["raw_datetime"], format="%Y%m%d %H%M%S")

start_date = "2016-11-07"   # 원하는 날짜로 바꿔 쓰면 됨
df = df[df["datetime"] >= start_date]

# 원래 raw_datetime 칼럼 제거
df = df.drop(columns=["raw_datetime"])

# 컬럼 순서 정리
df = df[["datetime", "open", "high", "low", "close", "volume"]]


# 숫자형으로 변환
df[["open", "high", "low", "close", "volume"]] = df[["open", "high", "low", "close", "volume"]].astype(float)

df=df.reset_index(drop=True)

In [14]:
df

,datetime,open,high,low,close,volume
0,2016-11-07 00:02:00,2105.50,2105.75,2105.50,2105.75,0.0
1,2016-11-07 00:03:00,2105.75,2106.00,2105.75,2106.00,0.0
2,2016-11-07 00:04:00,2106.25,2106.25,2106.00,2106.00,0.0
3,2016-11-07 00:05:00,2105.75,2106.00,2105.75,2105.75,0.0
4,2016-11-07 00:12:00,2106.00,2106.00,2106.00,2106.00,0.0
...,...,...,...,...,...,...
37754,2016-12-30 16:51:00,2234.00,2234.50,2234.00,2234.50,0.0
37755,2016-12-30 16:53:00,2234.50,2234.50,2234.50,2234.50,0.0
37756,2016-12-30 16:54:00,2234.50,2234.75,2234.25,2234.25,0.0
37757,2016-12-30 16:55:00,2234.00,2234.00,2233.75,2233.75,0.0


In [17]:
import pandas as pd

# 원본 CSV 불러오기 (세미콜론 구분)
df_18 = pd.read_csv(r"C:\Users\jinuk\Downloads\DAT_ASCII_SPXUSD_M1_2018\DAT_ASCII_SPXUSD_M1_2018.csv", sep=";", header=None,
                 names=["raw_datetime", "open", "high", "low", "close", "volume"])

# datetime 분리 및 변환
# "20160103 180100" → "2016-01-03 18:01:00"
df_18["datetime"] = pd.to_datetime(df_18["raw_datetime"], format="%Y%m%d %H%M%S")


# 원래 raw_datetime 칼럼 제거
df_18 = df_18.drop(columns=["raw_datetime"])

# 컬럼 순서 정리
df_18 = df_18[["datetime", "open", "high", "low", "close", "volume"]]


# 숫자형으로 변환
df_18[["open", "high", "low", "close", "volume"]] = df_18[["open", "high", "low", "close", "volume"]].astype(float)

df_18=df_18.reset_index(drop=True)

In [18]:
df_18

,datetime,open,high,low,close,volume
0,2018-01-01 16:30:00,2668.000,2668.00,2668.000,2668.00,0.0
1,2018-01-01 18:00:00,2675.000,2676.75,2674.500,2676.50,0.0
2,2018-01-01 18:01:00,2676.750,2677.75,2676.750,2677.75,0.0
3,2018-01-01 18:02:00,2677.500,2678.50,2677.500,2678.50,0.0
4,2018-01-01 18:03:00,2678.250,2678.25,2678.000,2678.25,0.0
...,...,...,...,...,...,...
310376,2018-12-31 16:09:00,2508.069,2510.07,2507.770,2510.07,0.0
310377,2018-12-31 16:10:00,2510.070,2510.27,2509.270,2509.57,0.0
310378,2018-12-31 16:11:00,2509.570,2509.77,2508.769,2509.27,0.0
310379,2018-12-31 16:12:00,2509.270,2509.57,2509.069,2509.27,0.0


In [27]:
import pandas as pd

# 원본 CSV 불러오기 (세미콜론 구분)
df_19 = pd.read_csv(r"C:\Users\jinuk\Downloads\DAT_ASCII_SPXUSD_M1_2019\DAT_ASCII_SPXUSD_M1_2019.csv", sep=";", header=None,
                 names=["raw_datetime", "open", "high", "low", "close", "volume"])

# datetime 분리 및 변환
# "20160103 180100" → "2016-01-03 18:01:00"
df_19["datetime"] = pd.to_datetime(df_19["raw_datetime"], format="%Y%m%d %H%M%S")

end_date = "2019-09-16"   # 원하는 날짜로 바꿔 쓰면 됨
df_19 = df_19[df_19["datetime"] <= end_date]

# 원래 raw_datetime 칼럼 제거
df_19 = df_19.drop(columns=["raw_datetime"])

# 컬럼 순서 정리
df_19 = df_19[["datetime", "open", "high", "low", "close", "volume"]]


# 숫자형으로 변환
df_19[["open", "high", "low", "close", "volume"]] = df_19[["open", "high", "low", "close", "volume"]].astype(float)

df_19=df_19.reset_index(drop=True)

In [28]:
import pandas as pd

# 세로로 쌓기
df_all = pd.concat([df, df_17, df_18, df_19], axis=0)

# 인덱스 새로 부여
df_all = df_all.reset_index(drop=True)


In [32]:
df_17

,datetime,open,high,low,close,volume
0,2017-01-02 18:00:00,2241.00,2244.50,2241.00,2243.50,0
1,2017-01-02 18:01:00,2243.75,2243.75,2243.00,2243.00,0
2,2017-01-02 18:02:00,2243.25,2243.25,2243.00,2243.25,0
3,2017-01-02 18:03:00,2243.00,2243.00,2243.00,2243.00,0
4,2017-01-02 18:04:00,2243.25,2243.75,2243.25,2243.75,0
...,...,...,...,...,...,...
222021,2017-12-29 16:53:00,2669.50,2669.75,2669.25,2669.25,0
222022,2017-12-29 16:54:00,2669.00,2669.25,2669.00,2669.00,0
222023,2017-12-29 16:55:00,2668.75,2668.75,2668.00,2668.25,0
222024,2017-12-29 16:57:00,2667.75,2668.50,2667.75,2668.00,0


In [30]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790517 entries, 0 to 790516
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   datetime  790517 non-null  datetime64[ns]
 1   open      790517 non-null  float64       
 2   high      790517 non-null  float64       
 3   low       790517 non-null  float64       
 4   close     790517 non-null  float64       
 5   volume    790517 non-null  float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 36.2 MB


In [31]:
df_all.to_excel("SPXUSD_filtered.xlsx")

In [ ]:
df_all.to_excel("SPXUSD_filtered.xlsx")